In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
# 1. Load, chunk and index the contents of the blog to create a retriever.

vectorstore = Chroma(persist_directory='../data/chroma_db', embedding_function=OpenAIEmbeddings())

In [5]:
retriever = vectorstore.as_retriever()

In [6]:
# 2. Incorporate the retriever into a question-answering chain.
### Contextualize question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [7]:
response = rag_chain.invoke({"input": "실업자도 지역별 노동조합에 가입할 수 있나요?"})
response["answer"]

'실업자나 구직중인 자도 노동조합에 가입이 가능합니다. 대법원은 일시적으로 실업 상태에 있는 자나 구직중인 자도 노동3권을 보장할 필요성이 있는 한 노동조합의 범위에 포함된다고 판결하였습니다. 따라서, 지역별 노동조합이 구직중인 실업자를 포함시킬 경우 가입이 가능합니다.'

# Adding chat history

In [8]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [9]:
# OK
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [12]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "실업자도 지역별 노동조합에 가입할 수 있나요?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "그때 최저임금은 어떻게 될까요?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=second_question),
        AIMessage(content=ai_msg_2["answer"]),
    ]
)

print(ai_msg_2["answer"])
print(chat_history)

최저임금은 국가에서 임금의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도이며, 사용자는 최저임금액 이상을 지급하여야 합니다. 현재 고용노동부에서 고시한 2016년 1월 1일부터 2016년 12월 31일까지 적용되는 최저임금액은 시간당 6,030원으로 정해져 있습니다.
[HumanMessage(content='실업자도 지역별 노동조합에 가입할 수 있나요?'), AIMessage(content='실업자도 노동조합에 가입할 수 있습니다. 대법원 판례에 따르면, 노동조합법에서 근로자로 보호되는 범위에는 일시적으로 실업 상태에 있는 자나 구직 중인 자도 포함되기 때문입니다. 따라서, 실업자도 노동조합에 가입할 수 있는 것으로 보입니다.'), HumanMessage(content='실업자도 지역별 노동조합에 가입할 수 있나요?'), AIMessage(content='최저임금은 국가에서 임금의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도이며, 사용자는 최저임금액 이상을 지급하여야 합니다. 현재 고용노동부에서 고시한 2016년 1월 1일부터 2016년 12월 31일까지 적용되는 최저임금액은 시간당 6,030원으로 정해져 있습니다.')]


아래는 다른 방식 (대화 연결을 더 쉽게 하는 방식이라고 함)

In [15]:
### Statefully manage chat history ###

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_message_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [16]:
conversational_rag_chain.invoke(
    {"input": "실업자도 지역별 노동조합에 가입할 수 있나요?"},
    config={
        "configurable": {"session_id": "abc123"}
        # constructs a key "abc123" in `store`.
    },
)["answer"]

'네, 지역별 노동조합은 구직 중인 실업자도 포함하여 가입이 가능합니다. 노동조합에는 특정 사용자에게 고용되어 있거나 구직 중인 자도 포함될 수 있으며, 근로자의 노동3권을 보장하는 것이 중요하다고 합니다. 따라서 실업 중인 경우에도 지역별 노동조합에 가입할 수 있습니다.'

In [17]:
conversational_rag_chain.invoke(
    {"input": "그때 최저임금은 어떻게 될까요?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'죄송합니다, 최저임금은 시간당 금액으로 정해지며, 그 금액은 해당 국가의 법률 및 정부 정책에 따라 매년 변경됩니다. 최저임금은 근로자의 생계비, 유사근로자의 임금, 노동생산성, 소득분배율 등을 고려하여 결정되며, 법률에 따라 최저임금심의위원회가 심의하여 결정됩니다.따라서 특정 연도의 최저임금은 해당 연도의 법률 및 정책에 따라 달라질 수 있습니다.'

In [18]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: 실업자도 지역별 노동조합에 가입할 수 있나요?

AI: 네, 지역별 노동조합은 구직 중인 실업자도 포함하여 가입이 가능합니다. 노동조합에는 특정 사용자에게 고용되어 있거나 구직 중인 자도 포함될 수 있으며, 근로자의 노동3권을 보장하는 것이 중요하다고 합니다. 따라서 실업 중인 경우에도 지역별 노동조합에 가입할 수 있습니다.

User: 그때 최저임금은 어떻게 될까요?

AI: 죄송합니다, 최저임금은 시간당 금액으로 정해지며, 그 금액은 해당 국가의 법률 및 정부 정책에 따라 매년 변경됩니다. 최저임금은 근로자의 생계비, 유사근로자의 임금, 노동생산성, 소득분배율 등을 고려하여 결정되며, 법률에 따라 최저임금심의위원회가 심의하여 결정됩니다.따라서 특정 연도의 최저임금은 해당 연도의 법률 및 정책에 따라 달라질 수 있습니다.



# Agents
답변을 예측하기 어렵다는 단점이 있지만 맥락을 위와 같이 주지 않아도 된다고 한다.<br>
컨텍스트화를 명시적으로 구축할 필요 없이 검색기에 대한 입력을 직접 생성

In [7]:
# retrieval tool

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "law_qna_retriever",
    "Searches and returns exceprts from the Autonomous Agents Q&A.",
)
tools = [tool]

In [22]:
tool.invoke("실업자도 지역별 노동조합에 가입할 수 있나요?")

'경우에는 원래부터 일정한 사용자에의 종속관계를 조합원의 자격요건으로 하는 것이 아닌 점에 비추어, 노조법 제2조 제1호 및 제4호 (라)목 본문에서 말하는 ‘근로자’에는 특정한 사용자에게 고용되어 현실적으로 취업하고 있는 자뿐만 아니라, 일시적으로 실업 상태에 있는 자나 구직중인 자도 노동3권을 보장할 필요성이 있는 한 그 범위에 포함된다.”라고 하였습니다(대법원 2004. 2. 27. 선고 2001두8568 판결).이에 비추어 볼때 지역별 노동조합이 그 구성원으로 구직중인 실업자도 포함시키도록 하고 있다면, 귀하도 노동조합에 가입이 가능할 것으로 보입니다."\n\n"동네 수퍼마켓들이 모여 협동조합을 운영해 왔는데, 성과가 매우 좋아 다른 지역의 수퍼마켓협동조합들과의 연합회 설립도 추진하려 합니다. 협동조합연합회는 어떻게 설립하면 되나요?",,"현대 자본주의 시장경제 구조에서 대기업과의 경쟁 및 사업역량 강화를 위해 ""협동조합 또는 사회적 협동조합 사이의 협동""은 선택이 아니라 필수라고 할 수 있으며, 같은 유형의 협동조합 또는 사회적협동조합들의 연합회 구성은 그러한 협동방법 중 하나입니다.협동조합연합회의 신규설립은 발기인 모집 → 정관 작성 → 설립동의자 모집 → 창립총회 개최 → 설립신고 → 발기인의 이사장에 대한 사무인계 → 조합원의 출자금 등 납입 → 설립등기의 단계를 거쳐 진행됩니다."\n"주택연금에 가입하려는데 가입 시 어떤 비용이 얼마나 드는지, 가입 후에 추가로 드는 비용이 있는지 궁금합니다.",,"주택연금 가입 시 가입자는 초기보증료, 근저당권 설정을 위한 법무사 비용, 등록면허세 및 지방교육세 등 세금, 대출기관 인지세(주택감정평가를 요청한 경우), 감정평가 비용 등의 비용을 부담합니다. 주택연금 가입이후에는 대출잔액에 대해 매월 일정 비율의 연보증료를 부담합니다."\n불법체류인 상태로 공장에서 근무하다가 다쳤는데 회사에서 병원비를 줄 수 없다고 합니다. 「산업재해보상보험법」의 요양급여를 받을 수 있나요?,,"불법체류 외국인근로자도 근로

In [8]:
# Agent constructor

from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

In [30]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HKs7ixXLq3o1uh0kYFiZLzR1', 'function': {'arguments': '{"query":"Task Decomposition"}', 'name': 'law_qna_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 71, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-670a0284-aa0c-4ad5-9672-6d2be4bbcd5a-0', tool_calls=[{'name': 'law_qna_retriever', 'args': {'query': 'Task Decomposition'}, 'id': 'call_HKs7ixXLq3o1uh0kYFiZLzR1'}])]}}
----
{'tools': {'messages': [ToolMessage(content='있는 권한이 부여됩니다.     결국 도시?군계획시설결정과 실시계획인가는 도시?군계획시설사업을 위하여 이루어지는 단계적 행정절차에서 별도의 요건과 절차에 따라 별개의 법률효과를 발생시키는 독립적인 행정처분입니다. 그러므로 선행처분인 도시?군계획시설결정에 하자가 있더라도 그것이 당연무효가 아닌 한 원칙적으로 후행처분인 실시계획인가에 승계되지 않는다고 할 것입니다(대법원 2017. 7. 18. 선고 2016두49938 판결참조).                   "\n\n한 건물의 구조에 따른 객관적인 용도에 의하여 결정됩니다. 따라서 구분건물에 관하여 구분소유가 성립될 당시 객

In [37]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, checkpointer=memory
)

In [33]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 93, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6a6ef3c6-76e8-4bff-ab90-a10b4b6544bc-0')]}}
----


In [34]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dz9OyZHOOHvVOIt4d6VwJ3cn', 'function': {'arguments': '{"query":"Task Decomposition"}', 'name': 'law_qna_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 117, 'total_tokens': 138}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1370866e-8003-460d-ba5d-db43652bc6d6-0', tool_calls=[{'name': 'law_qna_retriever', 'args': {'query': 'Task Decomposition'}, 'id': 'call_dz9OyZHOOHvVOIt4d6VwJ3cn'}])]}}
----
{'tools': {'messages': [ToolMessage(content='있는 권한이 부여됩니다.     결국 도시?군계획시설결정과 실시계획인가는 도시?군계획시설사업을 위하여 이루어지는 단계적 행정절차에서 별도의 요건과 절차에 따라 별개의 법률효과를 발생시키는 독립적인 행정처분입니다. 그러므로 선행처분인 도시?군계획시설결정에 하자가 있더라도 그것이 당연무효가 아닌 한 원칙적으로 후행처분인 실시계획인가에 승계되지 않는다고 할 것입니다(대법원 2017. 7. 18. 선고 2016두49938 판결참조).                   "\n\n한 건물의 구조에 따른 객관적인 용도에 의하여 결정됩니다. 따라서 구분건물에 관하여 구분소유가 성립될 당시

In [35]:
query = "What according to the blog post are common ways of doing it? redo the search"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_c4TKupJpAOuuvCnmg5pu88tY', 'function': {'arguments': '{"query":"Task Decomposition common ways"}', 'name': 'law_qna_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 2224, 'total_tokens': 2247}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-94f2384b-8dc5-4fdf-b7b1-f7fa19b7d568-0', tool_calls=[{'name': 'law_qna_retriever', 'args': {'query': 'Task Decomposition common ways'}, 'id': 'call_c4TKupJpAOuuvCnmg5pu88tY'}])]}}
----
{'tools': {'messages': [ToolMessage(content='구성하여 해당 일부공용부분의 관리를 이원적으로 할수 있도록 하고 있으나, 질의사례에서는 이원적으로 분리가 된 것이 아닌 입주자대표회의를 통해 일원적으로 관리되고 있는 것으로 추정됩니다. 상가와 아파트를\xa0일원적으로\xa0관리하고 있고, 상가건물 소유자 등으로부터\xa0건물의 유지 보수를 위한 장기수선충당금을 징수하고 있는 상황인 경우\xa0상가의 일부 공용부분에 대한 유지보수 비용은 상가 일부 공용부분임을 이유로 부담을 거부하는 것은 어려움이 있을 것으로 생각됩니다. 해당 부분의 갈등을 종국적으로 해결하기 위해서는 위와 같